In [7]:

import torch
from torch.utils.data import Dataset
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime
import pandas as pd
import torch
from torch.utils.data import Dataset
import numpy as np
from torch.utils.data import DataLoader
import os

'''
human keypoints: 34*2
court keypoints: 6*2
net keypoints:   4*2
ball keypoints:  1*2
total keypoints: 45*2

'''

class HitDataset(Dataset):
    def __init__(self, dataset_folder,num_consecutive_frames,normalization=True):
        self.dataset=[]
        self.positive=0
        self.negative=0
        # 遍历文件夹及其子文件夹，找到所有的CSV文件路径
        cnt=0
        for root, dirs, files in os.walk(dataset_folder):
            
            for file in files:
                
                if file.endswith(".csv"):
                    # 定义处理函数
                    data_path=os.path.join(root, file)
                    print(data_path)
                    
                    df= pd.read_csv(data_path, converters={"ball": eval,"top":eval,"bottom":eval,"court":eval,"net":eval})
                    
                    rows = len(df)
                    remainder = rows % 12
                    if remainder > 0:
                        num_to_pad = 12 - remainder
                    else:
                        num_to_pad = 0

                    if num_to_pad > 0:
                        last_row = df.iloc[-1]
                        padding_data = np.tile(last_row.values, (num_to_pad, 1))
                        padded_df = pd.DataFrame(padding_data, columns=df.columns)
                        df = pd.concat([df, padded_df], axis=0)
                        df = df.reset_index(drop=True)

                    small_dataset =df
                    
                    for i in range(len(small_dataset)):
                        if i%num_consecutive_frames!=0:
                            continue
                        if i>=len(small_dataset)-num_consecutive_frames:
                            break
                        oridata=small_dataset.loc[i:i+num_consecutive_frames-1,:].copy()
                        oridata=oridata.reset_index(drop=True)
                        data=[]
                        
                        target1=None
                        target2=[]
                        
                        for index,row in oridata.iterrows():
                            pos=np.array(row['pos'])
                            if str(pos)=='top': 
                                if target1 is None:
                                    target1=[0.0,1.0,0.0]
                                    self.positive+=1
                                target2.append(1)
                            elif str(pos)=='bottom':
                                if target1 is None:
                                    target1=[0.0,0.0,1.0]
                                    self.positive+=1
                                target2.append(1)
                            else:
                                target2.append(0)
                                    
                        
                            top=np.array(row['top']).reshape(-1,2)
                            bottom=np.array(row['bottom']).reshape(-1,2)
                            court=np.array(row['court']).reshape(-1,2)
                            net=np.array(row['net']).reshape(-1,2)
                            ball=np.array(row['ball']).reshape(-1,2)
                            
                            frame_data = np.concatenate((top, bottom, net,court, ball), axis=0)
                            
                            if normalization:
                                frame_data[:,0]/=1920
                                frame_data[:,1]/=1080
                            data.append(frame_data.reshape(1,-1))
                        data=np.array(data)
                        if target1 is None:
                            if self.negative>self.positive:
                                continue
                            target1=[1.0,0.0,0.0] 
                            self.negative+=1        
                        if 1 not in target2:
                            target2.append(1) 
                        else:
                            target2.append(0)               
                        self.dataset.append((data.reshape(-1),(target1,target2)))
    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, index):
        # 假设每个样本是一个元组 (input, target)
        sample = self.dataset[index]
        input_data = sample[0]
        target1 = sample[1][0]
        target2 = sample[1][1]
        
        # 转换为Tensor对象
        input_tensor = torch.tensor(input_data)
        target1_tensor = torch.tensor(target1)
        target2_tensor = torch.tensor(target2)

        return input_tensor, (target1_tensor,target2_tensor)
    
batch_size=30
num_consecutive_frames=12
shuffle=True
normalization=True

TrainDataset=HitDataset("./HitDataset/train",num_consecutive_frames,normalization)
ValidDataset=HitDataset("./HitDataset/valid",num_consecutive_frames,normalization)



./HitDataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-1.csv
./HitDataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-11.csv
./HitDataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-12.csv
./HitDataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-14.csv
./HitDataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-16.csv
./HitDataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-17.csv
./HitDataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-18.csv
./HitDataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-19.csv
./HitDataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022_Semi_finals\rally_1-2.csv
./HitDataset/train\Akane_YAMAGUCHI_AN_Se_Young_BWF_World_Championships_2022

In [8]:
class HitModel(nn.Module):
    def __init__(self, feature_dim, num_consecutive_frames):
        super(HitModel, self).__init__()
        self.num_consecutive_frames = num_consecutive_frames
        self.feature_dim = feature_dim
        self.gru1 = nn.GRU(feature_dim // num_consecutive_frames, 64, bidirectional=True, batch_first=True)
        self.gru2 = nn.GRU(128, 64, bidirectional=True, batch_first=True)
        self.global_maxpool = nn.MaxPool1d(num_consecutive_frames)
        self.pos = nn.Linear(128, 3)
        self.frame=nn.Linear(128,num_consecutive_frames+1)
        self.softmax = nn.Softmax(dim=-1)
        self.relu=nn.ReLU()

    def forward(self, x):
        batch_size = x.shape[0]
        x=x.float()
        x = x.view(batch_size, self.num_consecutive_frames, self.feature_dim // self.num_consecutive_frames)
        x, _ = self.gru1(x)
        x, _ = self.gru2(x)
        x = x.transpose(1, 2)
        x = self.global_maxpool(x).squeeze()
        pos= self.pos(x)
        frame=self.frame(x)
        pos = self.softmax(pos)
        frame=self.softmax(frame)
        return pos,frame

print(TrainDataset.positive,TrainDataset.negative)
print(ValidDataset.positive,ValidDataset.negative)

feature_dim=(17*2+6+4+1)*2*num_consecutive_frames
train_data_loader = DataLoader(TrainDataset, batch_size=batch_size, shuffle=shuffle)

valid_data_loader = DataLoader(ValidDataset, batch_size=batch_size, shuffle=shuffle)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model=HitModel(feature_dim,num_consecutive_frames)
criterion = nn.CrossEntropyLoss()
model.to(device)
criterion.to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 100
train_loss_list = []
test_loss_list=[]
best_model=None
best_accuracy=0
c1=1
c2=1
for epoch in range(num_epochs):
    train_loss_sum=0
    model.train()
    for batch_data in train_data_loader:
        inputs, labels = batch_data
        # print(inputs.shape)
        pos=labels[0]
        frame=labels[1]
        inputs = inputs.to(device)  # 将输入数据移动到设备上
        pos = pos.to(device).float()  # 将标签移动到设备上
        frame = frame.to(device).float()  # 将标签移动到设备上
        
        pos_pred,frame_pred = model(inputs)
        pos_pred=pos_pred.reshape(-1,3)
        frame_pred=frame_pred.reshape(-1,num_consecutive_frames+1)
        
        train_loss = c1*criterion(frame_pred,frame)+c2*criterion(pos_pred,pos)
        train_loss_sum+=train_loss.detach() 
        # 反向传播和优化
        optimizer.zero_grad()
        train_loss.backward()
        optimizer.step()
        # 执行你的训练或测试操作
    train_loss_list.append(train_loss_sum)

        
    # 打印训练信息
    if (epoch + 1) % 1== 0:
        print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch + 1, num_epochs,
                                                    train_loss.item()))
    with torch.no_grad():
        p_correct = 0
        f_correct=0
        p_total = 0
        for inputs, labels in valid_data_loader:
            pos=labels[0]
            frame=labels[1]
            inputs = inputs.to(device)  # 将输入数据移动到设备上
            frame = frame.to(device).float()  # 将标签移动到设备上
            pos = pos.to(device).float()  # 将标签移动到设备上

            # 前向传播
            pos_pred,frame_pred = model(inputs)
            # print(frame.shape)
            # print(pos.shape)
            pos_pred=pos_pred.reshape(-1,3)
            frame_pred=frame_pred.reshape(-1,num_consecutive_frames+1)
            
            p_true=torch.argmax(pos,axis=1)
            p_pred=torch.argmax(pos_pred,axis=1)
            f_true=torch.argmax(frame,axis=1)
            f_pred=torch.argmax(frame_pred,axis=1)
            
            p_total+=len(p_true)
            f_correct+=(f_true==f_pred).sum().item()
            p_correct+=(p_true==p_pred).sum().item()
        
        if p_correct/p_total>best_accuracy:
            best_accuracy=p_correct/p_total
            best_model=model
        try:
            print(f'Accuracy on test set for position predict: {p_correct/p_total}')
            print(f'Accuracy on test set for frame predict: {f_correct/p_total}')
        except:
            print("wrong")

17009 17009
1632 1629
Epoch [1/100], Loss: 3.0367
Accuracy on test set for position predict: 0.5752836553204539
Accuracy on test set for frame predict: 0.49954001839926404
Epoch [2/100], Loss: 3.1995
Accuracy on test set for position predict: 0.6176019625881631
Accuracy on test set for frame predict: 0.49954001839926404
Epoch [3/100], Loss: 2.8338
Accuracy on test set for position predict: 0.7056117755289788
Accuracy on test set for frame predict: 0.5283655320453848
Epoch [4/100], Loss: 2.8964
Accuracy on test set for position predict: 0.707145047531432
Accuracy on test set for frame predict: 0.5415516712664826
Epoch [5/100], Loss: 3.1745
Accuracy on test set for position predict: 0.7307574363692119
Accuracy on test set for frame predict: 0.5409383624655014
Epoch [6/100], Loss: 2.9005
Accuracy on test set for position predict: 0.7506899724011039
Accuracy on test set for frame predict: 0.5550444648880711
Epoch [7/100], Loss: 2.9975
Accuracy on test set for position predict: 0.7405703771

In [9]:
torch.save(best_model, 'hitdetectv2.pth')
print(best_accuracy)
model=best_model
with torch.no_grad():
    p_correct = 0
    f_correct=0
    p_total = 0
    for inputs, labels in valid_data_loader:
        pos=labels[0]
        frame=labels[1]
        inputs = inputs.to(device)  # 将输入数据移动到设备上
        frame = frame.to(device).float()  # 将标签移动到设备上
        pos = pos.to(device).float()  # 将标签移动到设备上

        # 前向传播
        pos_pred,frame_pred = model(inputs)
        # print(frame.shape)
        # print(pos.shape)
        pos_pred=pos_pred.reshape(-1,3)
        frame_pred=frame_pred.reshape(-1,num_consecutive_frames+1)
        
        p_true=torch.argmax(pos,axis=1)
        p_pred=torch.argmax(pos_pred,axis=1)
        f_true=torch.argmax(frame,axis=1)
        f_pred=torch.argmax(frame_pred,axis=1)
        
        p_total+=len(p_true)
        f_correct+=(f_true==f_pred).sum().item()
        p_correct+=(p_true==p_pred).sum().item()
    try:
        print(f'Accuracy on test set for position predict: {p_correct/p_total}')
        print(f'Accuracy on test set for frame predict: {f_correct/p_total}')
    except:
        print("wrong")

0.7951548604722478
Accuracy on test set for position predict: 0.7856485740570377
Accuracy on test set for frame predict: 0.5808034345292855


In [10]:
final_list=[]
true_list=[]
    
import pandas as pd
import matplotlib.pyplot as plt
import os
import random

special_path="ShuttleSet\ShuttleSet22\data4acreg\He_Bing_Jiao_Chen_Yu_Fei_GWANGJU_YONEX_Korea_Masters_2022_Final/rally_3-18.csv"

def random_csv_file(folder_path):
    csv_files = []
    
    # 遍历文件夹及其子文件夹，找到所有的CSV文件路径
    for root, dirs, files in os.walk(folder_path):
        for file in files:
            if file.endswith(".csv"):
                csv_files.append(os.path.join(root, file))
    
    # 随机选择一个CSV文件路径
    if len(csv_files) > 0:
        random_path = random.choice(csv_files)
        return random_path
    else:
        return None

# 指定文件夹路径
folder_path = "./HitDataset/test"

# 调用函数进行随机抽取
data_path = random_csv_file(folder_path)
# data_path="./HitDataset/test\Viktor_AXELSEN_Lakshya_SEN_YONEX_All_England_Open_2022_Finals/rally_1-31.csv"
df= pd.read_csv(data_path, converters={"ball": eval,"top":eval,"bottom":eval,"court":eval,"net":eval})

rows = len(df)
remainder = rows % 12
if remainder > 0:
    num_to_pad = 12 - remainder
else:
    num_to_pad = 0

if num_to_pad > 0:
    last_row = df.iloc[-1]
    padding_data = np.tile(last_row.values, (num_to_pad, 1))
    padded_df = pd.DataFrame(padding_data, columns=df.columns)
    df = pd.concat([df, padded_df], axis=0)
    df = df.reset_index(drop=True)

small_dataset =df
probs_list=[]
ipred_list=[]
itrue_list=[]
for i in range(len(small_dataset)):
    
    if i>=len(small_dataset)-num_consecutive_frames:
        break
    
    oridata=small_dataset.loc[i:i+num_consecutive_frames-1,:].copy()
    oridata=oridata.reset_index(drop=True)
    data=[]
    target1=None
    target2=[]
    

    if str(oridata.loc[0,'pos'])=='nan':
            true_list.append(0)
    elif str(oridata.loc[0,'pos'])=='top':
            true_list.append(1)
            itrue_list.append(i)
    elif str(oridata.loc[0,'pos'])=='bottom':
            true_list.append(2)
            itrue_list.append(i)
        
    
    for index,row in oridata.iterrows():
        pos=np.array(row['pos'])
        if str(pos)=='top': 
            if target1 is None:
                target1=[0.0,1.0,0.0]
            target2.append(1)
            
        elif str(pos)=='bottom':
            if target1 is None:
                target1=[0.0,0.0,1.0]
            target2.append(1)
        else:
            target2.append(0)
                
    
        top=np.array(row['top']).reshape(-1,2)
        bottom=np.array(row['bottom']).reshape(-1,2)
        court=np.array(row['court']).reshape(-1,2)
        net=np.array(row['net']).reshape(-1,2)
        ball=np.array(row['ball']).reshape(-1,2)
        
        frame_data = np.concatenate((top, bottom,net, court, ball), axis=0)
        
        if normalization:
            frame_data[:,0]/=1920
            frame_data[:,1]/=1080

        data.append(frame_data.reshape(1,-1))
    data=np.array(data).reshape(1,-1)
    if target1 is None:
        target1=[1.0,0.0,0.0] 
    
    # 前向传播
    pos_pred,frame_pred = model(torch.FloatTensor(data).to(device))
    
    pos_pred=pos_pred.reshape(-1,3)
    frame_pred=frame_pred.reshape(-1,num_consecutive_frames+1)
    p_pred=torch.argmax(pos_pred,axis=1).item()
    f_pred=torch.argmax(frame_pred,axis=1).item()
    if f_pred!=num_consecutive_frames:
        ipred_list.append(f_pred+i)
    final_list.append(p_pred)

print(true_list)
print(final_list)

print(itrue_list)

iipred_list=[]
fix=ipred_list[0]
bar=[]
for data in ipred_list:
    if data-fix<num_consecutive_frames:
        bar.append(data)
    else:
        if len(bar):
            iipred_list.append(int(sum(bar)/len(bar)))
        fix=data
        bar=[]
        bar.append(data)

if sum(bar)!=0:
    f=sum(bar)//len(bar)
    f=min(f,len(final_list)-1)
    iipred_list.append(f)
print(ipred_list)
print(iipred_list)
result=[final_list[i] for i in iipred_list]
print(result)    


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[12, 38]
[0, 1, 2, 3, 4, 5, 6, 35, 36, 35, 36, 35, 35, 35, 35, 36, 37, 35, 36, 45, 51, 50, 51, 52]
[3, 36, 47]
[2, 1, 2]
